# Tutorial: Dynamic Schema Selection with Schema Dict Pattern

**Category**: Schema Handlers  
**Difficulty**: Intermediate  
**Time**: 15-25 minutes

## Problem Statement

When building MCP servers or multi-tool systems, each tool requires its own schema for validation. Hardcoding validation logic for each tool creates maintenance burden and code duplication. You need a scalable pattern that routes tool invocations to the correct schema dynamically.

Consider an MCP server with tools like `search()`, `create()`, and `update()`. Each has different parameter structures. Manual dispatch logic becomes unwieldy:

```python
# ❌ Manual dispatch - doesn't scale
if tool == "search":
    schema = SearchSchema
elif tool == "create":
    schema = CreateSchema
elif tool == "update":
    schema = UpdateSchema
# ... 50 more tools
```

**Why This Matters**:
- **Scalability**: Adding new tools shouldn't require modifying dispatch logic
- **Type Safety**: Each tool's parameters should be validated against its specific schema
- **Code Reuse**: Generic processing pipeline should work for all tools

**What You'll Build**:
A production-ready schema registry pattern using a simple dict mapping (`{tool_name: schema}`) that enables dynamic schema selection, parameter mapping, and validation for multi-tool systems.

## Prerequisites

**Prior Knowledge**:
- Python dictionaries and Pydantic models
- Basic understanding of function call parsing
- Familiarity with tool/API dispatch patterns

**Required Packages**:
```bash
pip install lionherd-core  # >=0.1.0
```

**Optional Reading**:
- [API Reference: function_call_parser](../../../docs/api/libs/schema_handlers/function_call_parser.md)

In [1]:
# Standard library

# Third-party
from pydantic import BaseModel, Field

# lionherd-core - schema handlers
from lionherd_core.libs.schema_handlers import (
    map_positional_args,
    nest_arguments_by_schema,
    parse_function_call,
)

## Solution Overview

We'll implement a schema dict pattern for dynamic tool routing:

1. **Define Tool Schemas**: Create Pydantic models for each tool's parameters
2. **Build Schema Dict**: Map tool names to schemas (`{'search': SearchSchema}`)
3. **Parse Function Calls**: Extract tool name and arguments from call syntax
4. **Dynamic Selection**: Use tool name to select schema from dict
5. **Process with Schema**: Map args, nest structure, validate

**Key lionherd-core Components**:
- `parse_function_call()`: Parses Python function syntax into tool + arguments
- `map_positional_args()`: Maps positional args to parameter names
- `nest_arguments_by_schema()`: Restructures flat args into nested format

**Flow**:
```
Function Call String → Parse → Extract Tool Name → Select Schema → Map Args → Validate
   search("AI", 10)       ↓         "search"           SearchSchema    {query, limit}   ✓
```

**Expected Outcome**: A reusable pattern where adding a new tool only requires defining its schema and adding one entry to the schema dict.

### Step 1: Define Tool Schemas

Start by defining Pydantic schemas for different tools. Each schema represents the expected parameters for that tool.

**Why Separate Schemas**: Different tools have different parameter structures. `search()` needs `query` and `limit`, while `create()` needs `title` and `content`.

In [2]:
# Schema 1: Search tool
class SearchSchema(BaseModel):
    """Schema for search tool - finds items matching query."""

    query: str = Field(description="Search query string")
    limit: int = Field(default=10, description="Maximum results to return")
    category: str | None = Field(default=None, description="Filter by category")


# Schema 2: Create tool
class CreateSchema(BaseModel):
    """Schema for create tool - creates new items."""

    title: str = Field(description="Item title")
    content: str = Field(description="Item content/body")
    tags: list[str] = Field(default_factory=list, description="Item tags")


# Schema 3: Update tool
class UpdateSchema(BaseModel):
    """Schema for update tool - updates existing items."""

    item_id: str = Field(description="ID of item to update")
    title: str | None = Field(default=None, description="New title")
    content: str | None = Field(default=None, description="New content")


print("Defined 3 tool schemas:")
print(f"  SearchSchema: {list(SearchSchema.model_fields.keys())}")
print(f"  CreateSchema: {list(CreateSchema.model_fields.keys())}")
print(f"  UpdateSchema: {list(UpdateSchema.model_fields.keys())}")

Defined 3 tool schemas:
  SearchSchema: ['query', 'limit', 'category']
  CreateSchema: ['title', 'content', 'tags']
  UpdateSchema: ['item_id', 'title', 'content']


**Notes**:
- **Field descriptions**: Help with auto-generated documentation and LLM prompting
- **Optional fields**: Use `| None` with defaults for parameters that aren't always required
- **Validation**: Pydantic automatically validates types, constraints, and provides error messages

### Step 2: Create Schema Dict (Registry)

The core pattern: a simple dictionary mapping tool names to their schemas. This is your "schema registry" - simple yet powerful.

**Pattern**: `{tool_name: schema_class}` enables dynamic schema lookup in O(1) time.

In [3]:
# Schema registry - the core pattern
TOOL_SCHEMAS = {
    "search": SearchSchema,
    "create": CreateSchema,
    "update": UpdateSchema,
}

# Also store parameter order for each tool (used for positional arg mapping)
TOOL_PARAM_NAMES = {
    "search": ["query", "limit", "category"],
    "create": ["title", "content", "tags"],
    "update": ["item_id", "title", "content"],
}

print("Schema Registry:")
for tool_name, schema_cls in TOOL_SCHEMAS.items():
    param_count = len(schema_cls.model_fields)
    print(f"  '{tool_name}' → {schema_cls.__name__} ({param_count} fields)")

print("\nAdding a new tool:")
print("  1. Define schema (e.g., DeleteSchema)")
print("  2. Add to dict: TOOL_SCHEMAS['delete'] = DeleteSchema")
print("  3. Add params: TOOL_PARAM_NAMES['delete'] = ['item_id']")
print("  4. Done! Generic processing handles the rest.")

Schema Registry:
  'search' → SearchSchema (3 fields)
  'create' → CreateSchema (3 fields)
  'update' → UpdateSchema (3 fields)

Adding a new tool:
  1. Define schema (e.g., DeleteSchema)
  2. Add to dict: TOOL_SCHEMAS['delete'] = DeleteSchema
  3. Add params: TOOL_PARAM_NAMES['delete'] = ['item_id']
  4. Done! Generic processing handles the rest.


**Notes**:
- **Scalability**: Adding 100 tools is just 100 dict entries - no logic changes
- **Type safety**: Each tool gets validated against its specific schema
- **Parameter names**: Needed for mapping positional args (e.g., `search("AI")` → `{query: "AI"}`)

### Step 3: Parse Function Calls

Function calls come in as strings (from LLMs, user input, config files). Parse them to extract the tool name and arguments.

**Parsing**: Converts `search("AI news", 10)` → `{tool: "search", arguments: {_pos_0: "AI news", _pos_1: 10}}`

In [4]:
# Example function call strings (from LLM, user, config)
call_examples = [
    'search("AI trends", 20)',  # Positional args
    'create("My Title", "Content here", tags=["ai", "ml"])',  # Mixed
    'update(item_id="123", title="New Title")',  # Keyword args
]

print("Parsing function calls:\n")
for call_str in call_examples:
    # Parse the call
    parsed = parse_function_call(call_str)

    print(f"Input:  {call_str}")
    print(f"Tool:   {parsed['tool']}")
    print(f"Args:   {parsed['arguments']}")
    print()

print("Note: Positional args are labeled _pos_0, _pos_1, etc.")
print("      These will be mapped to actual param names next.")

Parsing function calls:

Input:  search("AI trends", 20)
Tool:   search
Args:   {'_pos_0': 'AI trends', '_pos_1': 20}

Input:  create("My Title", "Content here", tags=["ai", "ml"])
Tool:   create
Args:   {'_pos_0': 'My Title', '_pos_1': 'Content here', 'tags': ['ai', 'ml']}

Input:  update(item_id="123", title="New Title")
Tool:   update
Args:   {'item_id': '123', 'title': 'New Title'}

Note: Positional args are labeled _pos_0, _pos_1, etc.
      These will be mapped to actual param names next.


**Notes**:
- **Positional args**: Labeled `_pos_0`, `_pos_1` because we don't know parameter names yet
- **Keyword args**: Already have correct names, kept as-is
- **Complex types**: Lists, dicts, nested structures are parsed automatically

### Step 4: Dynamic Schema Selection and Processing

Now use the tool name to select the correct schema from the registry and process the arguments.

**Pattern**: `schema = TOOL_SCHEMAS[tool_name]` → map args → validate

In [5]:
def process_tool_call(call_str: str) -> BaseModel:
    """Process function call with dynamic schema selection."""
    # Step 1: Parse call → get tool name and arguments
    parsed = parse_function_call(call_str)
    tool_name = parsed["tool"]
    arguments = parsed["arguments"]

    # Step 2: Select schema from registry
    if tool_name not in TOOL_SCHEMAS:
        raise ValueError(f"Unknown tool: {tool_name}")

    schema_cls = TOOL_SCHEMAS[tool_name]
    param_names = TOOL_PARAM_NAMES[tool_name]

    # Step 3: Map positional args to parameter names
    mapped_args = map_positional_args(arguments, param_names)

    # Step 4: Nest arguments by schema structure (if needed)
    nested_args = nest_arguments_by_schema(mapped_args, schema_cls)

    # Step 5: Validate with schema
    validated = schema_cls(**nested_args)

    return validated


# Test with different tools
print("Processing tool calls with dynamic schema selection:\n")

test_calls = [
    'search("AI trends", 20)',
    'create("Tutorial", "Content about schemas")',
    'update("item-123", title="Updated Title")',
]

for call_str in test_calls:
    result = process_tool_call(call_str)
    print(f"Call:   {call_str}")
    print(f"Schema: {type(result).__name__}")
    print(f"Valid:  {result}")
    print()

Processing tool calls with dynamic schema selection:

Call:   search("AI trends", 20)
Schema: SearchSchema
Valid:  query='AI trends' limit=20 category=None

Call:   create("Tutorial", "Content about schemas")
Schema: CreateSchema
Valid:  title='Tutorial' content='Content about schemas' tags=[]

Call:   update("item-123", title="Updated Title")
Schema: UpdateSchema
Valid:  item_id='item-123' title='Updated Title' content=None



**Notes**:
- **Generic processing**: Same code handles all tools - no tool-specific logic
- **Type safety**: Each result is properly typed (SearchSchema, CreateSchema, etc.)
- **Validation**: Pydantic ensures all required fields present and types correct

### Step 5: Nested Schema Handling

For complex tools with nested structures (e.g., filters, metadata), the nesting function automatically restructures flat arguments.

**Pattern**: Flat args → detect nested fields in schema → restructure automatically

In [6]:
# Define a tool with nested structure
class FilterOptions(BaseModel):
    """Filter options for advanced search."""

    date_from: str | None = None
    date_to: str | None = None
    category: str | None = None


class AdvancedSearchSchema(BaseModel):
    """Advanced search with nested filter options."""

    query: str
    limit: int = 10
    filters: FilterOptions  # Nested structure


# Add to registry
TOOL_SCHEMAS["advanced_search"] = AdvancedSearchSchema
TOOL_PARAM_NAMES["advanced_search"] = ["query", "limit", "date_from", "date_to", "category"]

# Call with flat arguments
flat_call = 'advanced_search("AI", 15, "2024-01-01", "2024-12-31", "tech")'
print(f"Flat call: {flat_call}\n")

# Parse and show argument transformation
parsed = parse_function_call(flat_call)
print("Step 1 - Parsed (positional):")
print(f"  {parsed['arguments']}\n")

mapped = map_positional_args(parsed["arguments"], TOOL_PARAM_NAMES["advanced_search"])
print("Step 2 - Mapped to param names:")
print(f"  {mapped}\n")

nested = nest_arguments_by_schema(mapped, AdvancedSearchSchema)
print("Step 3 - Nested by schema:")
print(f"  {nested}\n")

validated = AdvancedSearchSchema(**nested)
print("Step 4 - Validated:")
print(f"  {validated}")
print(f"  Filters: {validated.filters}")

Flat call: advanced_search("AI", 15, "2024-01-01", "2024-12-31", "tech")

Step 1 - Parsed (positional):
  {'_pos_0': 'AI', '_pos_1': 15, '_pos_2': '2024-01-01', '_pos_3': '2024-12-31', '_pos_4': 'tech'}

Step 2 - Mapped to param names:
  {'query': 'AI', 'limit': 15, 'date_from': '2024-01-01', 'date_to': '2024-12-31', 'category': 'tech'}

Step 3 - Nested by schema:
  {'query': 'AI', 'limit': 15, 'filters': {'date_from': '2024-01-01', 'date_to': '2024-12-31', 'category': 'tech'}}

Step 4 - Validated:
  query='AI' limit=15 filters=FilterOptions(date_from='2024-01-01', date_to='2024-12-31', category='tech')
  Filters: date_from='2024-01-01' date_to='2024-12-31' category='tech'


**Notes**:
- **Automatic nesting**: `date_from`, `date_to`, `category` automatically grouped under `filters`
- **Schema detection**: Nesting logic inspects schema structure to determine grouping
- **Flat call syntax**: Users can call with flat args, system handles nesting

## Complete Working Example

Here's a production-ready implementation showing the schema dict pattern in a complete 30-line snippet. This is copy-paste ready for your MCP server or multi-tool system.

**Features**:
- ✅ Schema dict registry for dynamic selection
- ✅ Generic processing pipeline
- ✅ Handles positional and keyword args
- ✅ Automatic nested structure handling
- ✅ Type-safe validation

In [7]:
"""
Production-ready schema dict pattern for dynamic tool routing.
Copy this cell for a complete working system.
"""

from pydantic import BaseModel, Field

from lionherd_core.libs.schema_handlers import (
    map_positional_args,
    nest_arguments_by_schema,
    parse_function_call,
)


# Define tool schemas
class SearchSchema(BaseModel):
    query: str
    limit: int = 10


class CreateSchema(BaseModel):
    title: str
    content: str


# Schema registry - the core pattern
SCHEMAS = {"search": SearchSchema, "create": CreateSchema}
PARAMS = {"search": ["query", "limit"], "create": ["title", "content"]}


def process_call(call_str: str) -> BaseModel:
    """Process any tool call with dynamic schema selection."""
    parsed = parse_function_call(call_str)
    tool = parsed["tool"]

    # Dynamic schema selection
    schema = SCHEMAS[tool]
    params = PARAMS[tool]

    # Process arguments
    args = map_positional_args(parsed["arguments"], params)
    args = nest_arguments_by_schema(args, schema)

    return schema(**args)


# Example usage
result1 = process_call('search("AI trends", 20)')
result2 = process_call('create("My Post", "Content here")')

print(f"Search result: {result1}")
print(f"Create result: {result2}")

print("\n✅ Complete pattern in ~30 lines!")

Search result: query='AI trends' limit=20
Create result: title='My Post' content='Content here'

✅ Complete pattern in ~30 lines!


## Production Considerations

### Error Handling

**What Can Go Wrong**:
1. **Unknown tool**: User calls a tool not in registry
2. **Validation errors**: Arguments don't match schema (wrong types, missing required fields)
3. **Parse errors**: Malformed function call syntax
4. **Too many positional args**: More positional args than parameters

**Handling**:
```python
from pydantic import ValidationError

def safe_process_call(call_str: str) -> tuple[bool, BaseModel | str]:
    """Process call with comprehensive error handling."""
    try:
        # Parse
        parsed = parse_function_call(call_str)
        tool = parsed["tool"]
        
        # Check if tool exists
        if tool not in TOOL_SCHEMAS:
            return (False, f"Unknown tool: {tool}. Available: {list(TOOL_SCHEMAS.keys())}")
        
        # Process
        schema = TOOL_SCHEMAS[tool]
        params = TOOL_PARAM_NAMES[tool]
        args = map_positional_args(parsed["arguments"], params)
        args = nest_arguments_by_schema(args, schema)
        
        # Validate
        result = schema(**args)
        return (True, result)
        
    except ValueError as e:
        # Parse errors or positional arg errors
        return (False, f"Parse error: {e}")
    except ValidationError as e:
        # Pydantic validation errors
        return (False, f"Validation error: {e}")
    except Exception as e:
        # Unexpected errors
        return (False, f"Unexpected error: {e}")
```

### Performance

**Scalability**:
- **Registry lookup**: O(1) dict lookup per tool call
- **Parsing**: O(n) where n = argument count (typically < 20)
- **Validation**: O(n) field validation
- **Total overhead**: < 1ms per call for typical tools

**Benchmarks** (approximate, 5 parameters):
- Parse function call: ~0.1-0.3ms
- Map positional args: ~0.05ms
- Nest arguments: ~0.1ms
- Pydantic validation: ~0.2-0.5ms
- **Total**: ~0.5-1.0ms per tool call

**Optimization**:
```python
# Cache parameter names if calling same tool repeatedly
from functools import lru_cache

@lru_cache(maxsize=128)
def get_param_names(tool: str) -> list[str]:
    """Cached parameter name lookup."""
    return TOOL_PARAM_NAMES[tool]
```

### Testing

**Unit Tests**:
```python
def test_schema_registry_lookup():
    """Test that all registered tools have schemas."""
    for tool in TOOL_SCHEMAS:
        assert tool in TOOL_PARAM_NAMES
        assert len(TOOL_PARAM_NAMES[tool]) > 0

def test_dynamic_selection():
    """Test dynamic schema selection."""
    result = process_call('search("test", 5)')
    assert isinstance(result, SearchSchema)
    assert result.query == "test"
    assert result.limit == 5

def test_validation_errors():
    """Test that validation errors are caught."""
    success, _ = safe_process_call('search()')
    assert not success  # Missing required 'query' field
```

**Integration Tests**:
- Test all registered tools have valid schemas
- Test parameter count matches schema field count
- Test nested schema handling for complex tools
- Test error messages are helpful for debugging

### Monitoring

**Key Metrics**:
- **Tool usage**: Track which tools are called most (optimize hot paths)
- **Validation failures**: % of calls that fail validation (indicates schema issues or user errors)
- **Unknown tools**: Track attempts to call unregistered tools (indicates missing features)

**Observability**:
```python
import time

def process_call_with_metrics(call_str: str) -> BaseModel:
    """Process call with metric emission."""
    start = time.time()
    
    try:
        parsed = parse_function_call(call_str)
        tool = parsed["tool"]
        
        # Emit tool usage metric
        metrics.increment(f"tool.call.{tool}")
        
        result = process_call(call_str)
        
        metrics.increment(f"tool.success.{tool}")
        return result
        
    except Exception as e:
        metrics.increment(f"tool.error.{tool}")
        raise
    finally:
        duration = time.time() - start
        metrics.timing("tool.duration", duration)
```

### Configuration Management

**Registry Organization**:
```python
# For large systems, organize by category
SEARCH_TOOLS = {"search": SearchSchema, "advanced_search": AdvancedSearchSchema}
CRUD_TOOLS = {"create": CreateSchema, "update": UpdateSchema, "delete": DeleteSchema}
ADMIN_TOOLS = {"backup": BackupSchema, "restore": RestoreSchema}

# Combine into main registry
TOOL_SCHEMAS = {**SEARCH_TOOLS, **CRUD_TOOLS, **ADMIN_TOOLS}
```

**Schema Versioning**:
```python
# Version schemas for backward compatibility
TOOL_SCHEMAS = {
    "search": SearchSchemaV2,  # Current version
    "search_v1": SearchSchemaV1,  # Legacy support
}
```

## Variations

### 1. Class-Based Registry

**When to Use**: Large systems with many tools, need validation and tooling support

**Approach**:
```python
class ToolRegistry:
    """Type-safe tool registry with validation."""
    
    def __init__(self):
        self._schemas: dict[str, type[BaseModel]] = {}
        self._params: dict[str, list[str]] = {}
    
    def register(self, name: str, schema: type[BaseModel], params: list[str]):
        """Register a tool with validation."""
        # Validate param count matches schema fields
        if len(params) != len(schema.model_fields):
            raise ValueError(f"Param count mismatch for {name}")
        
        self._schemas[name] = schema
        self._params[name] = params
    
    def process(self, call_str: str) -> BaseModel:
        """Process call using registered schemas."""
        parsed = parse_function_call(call_str)
        tool = parsed["tool"]
        
        if tool not in self._schemas:
            raise ValueError(f"Unregistered tool: {tool}")
        
        schema = self._schemas[tool]
        params = self._params[tool]
        
        args = map_positional_args(parsed["arguments"], params)
        args = nest_arguments_by_schema(args, schema)
        
        return schema(**args)

# Usage
registry = ToolRegistry()
registry.register("search", SearchSchema, ["query", "limit"])
registry.register("create", CreateSchema, ["title", "content"])

result = registry.process('search("AI", 10)')
```

**Trade-offs**:
- ✅ Type safety and validation at registration time
- ✅ Encapsulation and better tooling support
- ✅ Easier testing (mock registry)
- ❌ More boilerplate than simple dict
- ❌ Slightly more complex for small systems

### 2. Auto-Generate Param Names from Schema

**When to Use**: Want to avoid manually maintaining TOOL_PARAM_NAMES

**Approach**:
```python
def get_param_names(schema: type[BaseModel]) -> list[str]:
    """Extract parameter names from schema field order."""
    return list(schema.model_fields.keys())

# Simplified registry - only schemas needed
TOOL_SCHEMAS = {"search": SearchSchema, "create": CreateSchema}

def process_call(call_str: str) -> BaseModel:
    parsed = parse_function_call(call_str)
    tool = parsed["tool"]
    schema = TOOL_SCHEMAS[tool]
    
    # Auto-generate param names
    params = get_param_names(schema)
    
    args = map_positional_args(parsed["arguments"], params)
    args = nest_arguments_by_schema(args, schema)
    return schema(**args)
```

**Trade-offs**:
- ✅ Less maintenance (single source of truth)
- ✅ Impossible for param names to get out of sync
- ❌ Relies on Python 3.7+ dict ordering guarantee
- ❌ Can't customize param order (must match schema field order)

### 3. Decorator-Based Registration

**When to Use**: Want declarative, Django/Flask-style tool registration

**Approach**:
```python
TOOL_REGISTRY = {}

def tool(name: str, params: list[str]):
    """Decorator to register tool schema."""
    def decorator(schema_cls: type[BaseModel]):
        TOOL_REGISTRY[name] = {"schema": schema_cls, "params": params}
        return schema_cls
    return decorator

# Declarative registration
@tool("search", ["query", "limit"])
class SearchSchema(BaseModel):
    query: str
    limit: int = 10

@tool("create", ["title", "content"])
class CreateSchema(BaseModel):
    title: str
    content: str

# Process using registry
def process_call(call_str: str) -> BaseModel:
    parsed = parse_function_call(call_str)
    tool = parsed["tool"]
    
    tool_info = TOOL_REGISTRY[tool]
    schema = tool_info["schema"]
    params = tool_info["params"]
    
    args = map_positional_args(parsed["arguments"], params)
    args = nest_arguments_by_schema(args, schema)
    return schema(**args)
```

**Trade-offs**:
- ✅ Clean, declarative syntax
- ✅ Schema definition and registration in one place
- ❌ Magic (less explicit than dict)
- ❌ Harder to see full registry at a glance

## Choosing the Right Variation

| Scenario | Recommended Variation |
|----------|----------------------|
| < 10 tools, simple system | Base implementation (dict) |
| 10-50 tools, need validation | Class-Based Registry |
| Want minimal maintenance | Auto-Generate Param Names |
| Large codebase, want declarative style | Decorator-Based Registration |
| MCP server (production) | Class-Based Registry + Auto-Generate |

## Summary

**What You Accomplished**:
- ✅ Built a schema dict registry for dynamic tool routing
- ✅ Implemented generic processing pipeline that works for any tool
- ✅ Handled positional args, keyword args, and nested structures
- ✅ Created a 30-line production-ready pattern
- ✅ Learned error handling, testing, and monitoring strategies

**Key Takeaways**:
1. **Schema dict is the core pattern**: `{tool_name: schema}` enables O(1) dynamic selection
2. **Generic processing scales**: Same code handles 5 tools or 500 tools
3. **Positional arg mapping**: Requires parameter name list per tool
4. **Nested structures are automatic**: `nest_arguments_by_schema()` detects and restructures
5. **Adding tools is trivial**: Define schema + add dict entry = done

**When to Use This Pattern**:
- ✅ Building MCP servers with multiple tools
- ✅ Multi-tool API dispatchers
- ✅ LLM function calling systems
- ✅ Config-driven command processors
- ✅ Any system where tools are added/removed frequently
- ❌ Single-tool systems (overkill - use schema directly)
- ❌ Fixed tool set that never changes (consider hardcoded dispatch)

## Related Resources

**lionherd-core API Reference**:
- [function_call_parser](../../../docs/api/libs/schema_handlers/function_call_parser.md) - Complete API documentation

**Reference Notebooks**:
- [Schema Handlers Patterns](../references/schema_handlers.ipynb) - Additional schema handling techniques

**Related Tutorials**:
- [Fuzzy Validation](../ln_utilities/fuzzy_validation.ipynb) - Handle schema field name variations

**External Resources**:
- [Pydantic Documentation](https://docs.pydantic.dev/) - Schema validation and models
- [MCP Specification](https://spec.modelcontextprotocol.io/) - Model Context Protocol for tool servers
- [Python AST Module](https://docs.python.org/3/library/ast.html) - Abstract syntax tree parsing (used by parser)